# Pull And Run Docker Image
## Using Triton and Bria-attribution-model

In [ ]:

import os
import requests

api_token = '<API_TOKEN>'

def set_repository_access_credentials():
    """Fetch ECR credentials from localhost endpoint"""
    try:
        headers = {"api_token":  api_token}
        url ="https://engine.prod.bria-api.com" #"http://localhost:5000" 
        response = requests.post(f"{url}/v2/image/attribution/ecr_credentials", headers=headers)
        response.raise_for_status()
        credentials = response.json()["result"]
        
        # Set environment variables
        os.environ['AWS_ACCESS_KEY_ID'] = credentials['access_key_id']
        os.environ['AWS_SECRET_ACCESS_KEY'] = credentials['secret_access_key']
        os.environ['AWS_SESSION_TOKEN'] = credentials['session_token']
        os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'
        
        print(f"Successfully fetched ECR credentials, expires at: {credentials['expiration']}")
        return credentials
    except Exception as e:
        print(f"Failed to fetch ECR credentials: {e}")
        raise

# Fetch ECR credentials from localhost endpoint
credentials = set_repository_access_credentials()

# Login to ECR 
! aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 300465780738.dkr.ecr.us-east-1.amazonaws.com

# Pull images (unlimited times)
! docker pull 300465780738.dkr.ecr.us-east-1.amazonaws.com/bria-attribution-embeddings:latest
! docker kill $(docker ps -q --filter expose=8002)
! docker run -d   --runtime=nvidia   --shm-size=16gb   -p 8000:8000   -p 8001:8001   -p 8002:8002 300465780738.dkr.ecr.us-east-1.amazonaws.com/bria-attribution-embeddings:latest

Successfully fetched ECR credentials, expires at: 2025-08-27T19:49:21+00:00
Unable to locate credentials. You can configure credentials by running "aws configure".
Error: Cannot perform an interactive login from a non TTY device
latest: Pulling from bria-attribution-embeddings

e0ac0f23: Pulling fs layer 
2a7bc3a8: Pulling fs layer 
b700ef54: Pulling fs layer 
5783a0f9: Pulling fs layer 
ba0172c5: Pulling fs layer 
f9032791: Pulling fs layer 
e18001c2: Pulling fs layer 
5783a0f9: Waiting fs layer 
ba0172c5: Waiting fs layer 
d44ce3f2: Pulling fs layer 
f9032791: Waiting fs layer 
e18001c2: Waiting fs layer 
5c341569: Pulling fs layer 
d44ce3f2: Waiting fs layer 
5c3dbc29: Pulling fs layer 
980ccfd7: Pulling fs layer 
3d1f6652: Pulling fs layer 
c0ed6d6b: Pulling fs layer 
e4229993: Pulling fs layer 
0250d71b: Pulling fs layer 
1a2201f9: Pulling fs layer 
127b8cc7: Pulling fs layer 
3fac024: Waiting fs layer 
67ac549: Waiting fs layer 
c341569: Waiting fs layer 
aef015d: Waiting fs laye

# Client Side

## Installations

In [ ]:
import os
import uuid
import requests
import tritonclient.http as httpclient
from PIL import Image
from embedder import BRIAEmbedder

## Inference Request

In [ ]:
url = '<BRIA_EMBEDDINGS_URL>:8000'
image_path = '<IMAGE_PATH>'

triton_client = httpclient.InferenceServerClient(
    url=url,
    # ssl=True,
    # ssl_context_factory=gevent.ssl._create_default_https_context,
)
image = Image.open(image_path)
bria_pipeline = BRIAEmbedder(triton_client)
img_embeddings = bria_pipeline.run_on_image(image)


Unused or unrecognized kwargs: padding.


Sending request to bria_attribution_model...
{'bria_attribution_model_infer': 0.011281967163085938}


## Send Attribution API

In [ ]:
model_version = os.environ.get("model_version", "2.3")
attribution_endpoint = os.environ.get("attribution_endpoint", "httsp://engine.prod.bria-api.com/v2/image/attribution/by_embedding")
api_token = '<API-TOKEN>'

try:
    print(f"Read data from queue")
    embeddings_uid = str(uuid.uuid4())
    print(f"embeddings_uid: {embeddings_uid}")
    request_body = {
        "embeddings_base64": img_embeddings,
        "embeddings_uid": embeddings_uid,
        "model_version": 2.3,
        "agent":  api_token,
        "model_name": 'replace-backgroud',
    }
    
    headers = {
        "api_token":  api_token
    }

    if attribution_endpoint:
        response = requests.post(attribution_endpoint, json=request_body, headers=headers)
        if response.status_code != 202 or response.status_code != 200:
            print(f"API request failed with status code {response.status_code}: {response.text}")
            raise Exception(f"API request failed with status code {response.status_code}: {response.text}")
        else:
            print(f"Request body: {request_body}")
except Exception as ex:
    print(f"Error processing record: {ex}")
    raise

Read data from queue
embeddings_uid: 18ebbd3f-511e-4652-ba8d-1ad58a6d43ce
Request body: {'embeddings_base64': ['ltHGPvhMQT8BIse+kEWcPmjtpTuCk4O+ERtPPo7IHD8sXMk+H4irPvl6ID6g7ZC8eseWvs7v6b0Y390+PlJvvktrA8CPes29C4hIPsTi6z6u9zg+sDZNvsII/b3GFQY+ZrmIPmI4oT6S3uS9g4QNPnEVK75w7lC+4s6LPkUb5b1w/Mm+VaoWvr2Wlj4Xivk++rONvZq6Hj799zc+dKFfPtiqoD1OHtU+kOl0veBpZ7yVArY+w06TPgBQxz5gjUy86O5hPuSfSL3VprE9wYS5Pod7Zz7GD2g+WvTzvkVOYD+WWNS+GL1aPn25Oj540Sk/appcv+iAoL3ut7s+BUNJPibbT74ggH+9OUxpvtoCFT8gXSY+yhkVvpU9Or7KGZM9aMCiPaxSPb4R92G+MNcuv1By/rzuAcK+Do6lvbc8CL+9S6q+ndFvvqZKFj6LHgq/1USDPnTn3bzIiPm+IBLsvfCHLL/JDQi/I+gCPtcPzb4gDvrA6cZav+2ftD6gnIo+goxWPpYWnb24wd69fCKzv0Tu5zwldXi+y+KIvgA8OTzuUoU9UIyXO/mJs74uKzy9jOaTvpAbWT0L7Sc+MaMFv7Rtgj3aZK++fAkgPa/rNL5gIxA8t/llvrbKyT7Mo5++EtJ7vpiqWT6nJJY+Eg95vgkPGr7wRle+c+8Vvl4PWT52V4K+zElXPkBrMLxINfi9YjZAPvCcej+6+Qg+DiAMPhzkf70Ckya/iupzvp47Qz5AdM+9jH5cvUBLKz6iA80+bDwNvUa0Vj7vAWo+cD1rPkkU2b5q8KI+8FcpPUb7eD41XZI/OiwAv9CCQr47LxK/QzccP3RGXb5y1ZO+TfIiPshnqL0gb9a7WOyOvXhYlDxEDs+9jdYQvmws